**All Tables STRUCTURES**

```console
Table marketing_spend {
  id            int         [pk, increment] // surrogate primary key
  date          date
  channel       varchar
  campaign      varchar
  spend         decimal
  impressions   int
  clicks        int
  installs      int
}

Table users {
  user_id       varchar     [pk]
  installed_at  timestamp
  created_at    timestamp
  updated_at    timestamp
  channel       varchar
  campaign      varchar
}

Table user_touchpoints {
  id              int       [pk, increment]
  user_id         varchar   [ref: > users.user_id]
  touchpoint_date timestamp
  channel         varchar
  campaign        varchar
  touchpoint_type varchar
  source          varchar
  medium          varchar
  conversion      boolean
}

Table revenue_and_rewards {
  id             int       [pk, increment]
  user_id        varchar   [ref: > users.user_id]
  date           date
  revenue        decimal
  reward_to_user decimal
}
```

In [ ]:
-- In psql, load the marketing_spend data:
\copy marketing_spend(id,date, channel, campaign, spend, impressions, clicks, installs) FROM '/path/to/marketing_spend.csv' DELIMITER ',' CSV HEADER;

-- Load the users data:
\copy users(user_id, installed_at, created_at, updated_at, channel, campaign) FROM '/path/to/users.csv' DELIMITER ',' CSV HEADER;

-- Load the user_touchpoints data:
\copy user_touchpoints(id,user_id, touchpoint_date, channel, campaign, touchpoint_type, source, medium, conversion) FROM '/path/to/user_touchpoints.csv' DELIMITER ',' CSV HEADER;

-- Load the revenue_and_rewards data:
\copy revenue_and_rewards(id,user_id, date, revenue, reward_to_user) FROM '/path/to/revenue_and_rewards.csv' DELIMITER ',' CSV HEADER;


In [15]:
import psycopg2

# Connection Details
SUPABASE_USER = "postgres.ylhsfopeerhruicwrdsl"
SUPABASE_PASSWORD = "Alekhya@28"
SUPABASE_HOST = "aws-0-us-east-1.pooler.supabase.com"
SUPABASE_PORT = "5432"
SUPABASE_DBNAME = "postgres"

# Connect to the database
try:
    connection = psycopg2.connect(
        user=SUPABASE_USER,
        password=SUPABASE_PASSWORD,
        host=SUPABASE_HOST,
        port=SUPABASE_PORT,
        dbname=SUPABASE_DBNAME
    )
    print("Connection successful!")
    
    # Create a cursor to execute SQL queries
    cursor = connection.cursor()
    
    # Select query
    cursor.execute("SELECT NOW();")
    result = cursor.fetchone()
    print("Current Time:", result)

    # Close the cursor and connection
    cursor.close()
    connection.close()
    print("Connection closed.")

except Exception as e:
    print(f"Failed to connect: {e}")

Connection successful!
Current Time: (datetime.datetime(2025, 2, 16, 14, 13, 6, 803105, tzinfo=datetime.timezone.utc),)
Connection closed.


In [ ]:
import psycopg2
import csv

# Connection Details
SUPABASE_USER = "postgres.ylhsfopeerhruicwrdsl"
SUPABASE_PASSWORD = "Alekhya@28"
SUPABASE_HOST = "aws-0-us-east-1.pooler.supabase.com"
SUPABASE_PORT = "5432"
SUPABASE_DBNAME = "postgres"

connection = psycopg2.connect(
        user=SUPABASE_USER,
        password=SUPABASE_PASSWORD,
        host=SUPABASE_HOST,
        port=SUPABASE_PORT,
        dbname=SUPABASE_DBNAME
    )
cursor = connection.cursor()

def load_csv_to_table(csv_file, table, columns):
    with open(csv_file, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        rows = [tuple(row[col] for col in columns) for row in reader]
    placeholders = ','.join(['%s'] * len(columns))
    query = f"INSERT INTO {table}({', '.join(columns)}) VALUES ({placeholders})"
    cursor.executemany(query, rows)
    connection.commit()

# Ingest data into each table
load_csv_to_table('marketing_spend.csv', 'marketing_spend', ['date', 'channel', 'campaign', 'spend', 'impressions', 'clicks', 'installs'])
load_csv_to_table('users.csv', 'users', ['user_id', 'installed_at', 'created_at', 'updated_at', 'channel', 'campaign'])
load_csv_to_table('user_touchpoints.csv', 'user_touchpoints', ['user_id', 'touchpoint_date', 'channel', 'campaign', 'touchpoint_type', 'source', 'medium', 'conversion'])
load_csv_to_table('revenue_and_rewards.csv', 'revenue_and_rewards', ['user_id', 'date', 'revenue', 'reward_to_user'])

cursor.close()
connection.close()
